### Setup

In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

from importlib import reload
reload(base)

In [ ]:
base_size = base.font_sizes['base_size']

sns.set_style('ticks')
sns.set_context('paper')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': base_size, 'ytick.labelsize': base_size})

In [ ]:
output_path = rd.rootdir/'output'/'fig_circuit-design'
cache_path = rd.rootdir/'output'/'controller-tuning-transfections'/'df_binned.gzip'

# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path).dropna()
display(data)

In [ ]:
# Create dicts to specify colors/markers
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
metadata['TS'] = metadata['ts_kind']
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']

In [ ]:
# Group and compute stats
stat_list = [np.mean, np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = data.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['marker'].count().reset_index()['marker']
stats = stats.merge(metadata, how='left', on='construct')
stats.sort_values(['design','ts_kind','ts_num'], inplace=True)

display(stats)

In [ ]:
stat_name = {'output_gmean': 'Geometric mean', 'output_std': 'Standard deviation', 'output_variation': 'Coefficient of variation'}

### Figure

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels.
fig = plt.figure(figsize=(7.5,4))
fig_gridspec = matplotlib.gridspec.GridSpec(3, 3, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[0.4,1,1], width_ratios=[1,3,1])
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:2]),
    'B': fig.add_subfigure(fig_gridspec[1,:2]),
    'C': fig.add_subfigure(fig_gridspec[2,:2]),
    'D': fig.add_subfigure(fig_gridspec[:,2]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}.', fontsize=base.font_sizes['subpanel_label'], fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

fig_path = output_path/'fig_circuit-design.pdf'
fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['B']
subfig.subplots_adjust(left=0.35, bottom=0.3, top=0.95, right=0.97)
axes = subfig.subplots(ncols=3, sharex=True, sharey=True, gridspec_kw={'wspace': 0.2})

for i,ax in enumerate(axes):
    plot_df = data[(data['miR']=='miR.FF5') & (data['promoter']=='CMV') & (data['biorep']==2) & (data['design']==i+1) & (data['output_norm']>1) & (data['ts']!='FF5x4')]
    sns.lineplot(data=plot_df, x='bin_marker_median_norm', y='output_norm', hue='construct', palette=construct_palette, 
                 style='construct', markers=construct_markers, ax=ax, legend=False, estimator=sp.stats.gmean, 
                 errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
    ax.set(xscale='log', yscale='log',xlabel='')
    sns.despine(ax=ax)
    ax.minorticks_on()
    ax.grid()
axes[0].set(xlabel='Normalized marker', ylabel='Normalized output')

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['C']
subfig.subplots_adjust(left=0.35, bottom=0.25, top=0.95, right=0.97)
axes = subfig.subplots(ncols=3, sharex=True, sharey=True, gridspec_kw={'wspace': 0.2})

for i,ax in enumerate(axes):
    plot_df = data[(data['miR']=='miR.FF5') & (data['promoter']=='CMV') & (data['biorep']==1) & (data['design']==i+1) & (data['ts']!='FF5x4')]
    sns.kdeplot(data=plot_df, x='output', hue='construct', palette=construct_palette, ax=ax, legend=False,
                fill=False, log_scale=True, common_norm=False)
    ax.set(xlabel='', xlim=(1e1, 1e6))
    sns.despine(ax=ax)
    ax.minorticks_off()
axes[0].set(xlabel='Output',)

fig.savefig(rd.outfile(fig_path))

In [ ]:
def generate_xticklabels(
    df_labels: pd.DataFrame,
    ax_col,
    label_cols,
    *,
    ax = None,
    annotate = True,
    align_ticklabels = "center",
    align_annotation = "right",
):
    # Draw plotting canvas to generate original xticklabels
    if ax is None:
        ax = plt.gca()
    ax.figure.canvas.draw()

    # Create dictionary from DataFrame, where keys are original xticklabels
    #  and values are dictionaries of (metadata_key, metadata_value) pairs
    dict_labels_by_xticklabel = df_labels.set_index(ax_col).to_dict(orient="index")

    # Loop over xticklabels and set new values
    ax_labels = []
    for item in ax.get_xticklabels():
        if item.get_text() in dict_labels_by_xticklabel:
            dict_labels = dict_labels_by_xticklabel[item.get_text()]

            # For each specified metadata key (label_cols), get the metadata value
            #  and concatenate all values into separate lines of a single string
            new_xticklabel = "\n".join([str(dict_labels[i]) for i in label_cols])
            ax_labels.append(new_xticklabel)
        else: ax_labels.append(item.get_text())

    ax.set_xticks(ax.get_xticks(), ax_labels, multialignment=align_ticklabels)

    # Get Artists for first axes labels
    xlabel_bbox = ax.get_xticklabels()[0]
    ylabel_bbox = ax.get_yticklabels()[0]

    font_size = plt.rcParams["xtick.labelsize"]

    # Annotate plot with metadata keys
    #   x value: align the right of the annotation bbox (ha='right')
    #       with the right (x=1) of the ylabel bbox (xcoord=ylabel_bbox)
    #   y value: align the vertical center of the annotation bbox (va='center')
    #       with the vertical center (y=0.5) of the xlabel bbox (ycoord=xlabel_bbox)
    if annotate:
        ax.annotate(
            text="\n".join(label_cols),
            xy=(1, 0.5),
            xycoords=(ylabel_bbox, xlabel_bbox),
            ha="right",
            va="center",
            multialignment=align_annotation,
            fontsize=font_size,
        )

In [ ]:
subfig = subfigures['D']
subfig.subplots_adjust(left=0.3, bottom=0.105, top=0.95, right=0.99)
axes = subfig.subplots(nrows=2, gridspec_kw={'hspace': 0.45})

stat_list = ['output_gmean', 'output_variation']
for i, stat in enumerate(stat_list):
    ax = axes[i]
    plot_df = stats[(stats['miR']=='miR.FF5') & (stats['promoter']=='CMV') & (stats['ts']!='FF5x4')]
    sns.scatterplot(data=plot_df, x='construct', y=stat, hue='construct', palette=construct_palette, 
                    style='construct', markers=construct_markers, ax=ax, legend=False)
    sns.despine(ax=ax)
    ax.set(xlabel='', ylabel='', title=stat_name[stat], yscale='log')
    if i==1: ax.set(yscale='linear',) #ylim=(0,ax.get_ylim()[1]))
    generate_xticklabels(metadata, 'construct', ['TS','design'], ax=ax, annotate=True)
    
fig.savefig(rd.outfile(fig_path))

In [ ]:
# Save to OneDrive
fig_path = rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'python-output'/'fig_circuit-design.pdf'
fig.savefig(rd.outfile(fig_path))